In [1]:
from pathlib import Path
import pandas as pd
import json

# On macOS, zip a file like so: `zip -r UserEnv.csv.zip UserEnv.csv`
# macOS default compression includes an unwanted dotfile `__MACOSX/.<filename>` which causes issues with pandas
MA_FS_ROOT = Path.home()
MA_PATH = Path("MALightingTechnology/gma3_library/search_plugin_results")
CSV_AS_ZIP = MA_FS_ROOT / MA_PATH / Path("UserEnv.csv.zip")

BAD_LINES = Path("foo_bad_lines.csv")
open(BAD_LINES, "w").close()


def write_bad_line(line):
    fp = open(BAD_LINES, "a")
    fp.write(f"{str(line)}\n")
    return "skip"  # tell callback to skip the line while processing


csv: pd.DataFrame = pd.read_csv(
    CSV_AS_ZIP,
    compression="zip",
    on_bad_lines=write_bad_line,
    engine="pyarrow",
)
csv.drop(columns=csv.columns[-1], inplace=True)

In [2]:
addresses: pd.Series = csv["Address"].drop_duplicates()
addresses.dropna(inplace=True)
addresses = addresses.astype(str)
addresses = addresses.apply(lambda x: x.replace('"', ""))
addresses = addresses.apply(lambda x: x.split("."))

In [3]:
def create_nested_dict(paths):
    """Source - GPT Prompt:
    generate a python script to create a nested dictionary based on a list of paths. this has to work similarly as to create a new path in a local filesystem with the command "mkdir -p"
    """

    _nested_dict = {}
    for _path in paths:
        _current_dict = _nested_dict
        for _component in _path:
            if _component not in _current_dict:
                _current_dict[_component] = {}
            _current_dict = _current_dict[_component]
    return _nested_dict


nested_dict = create_nested_dict(addresses)

In [4]:
path = "./foo.json"
with open(path, "w", encoding="utf-8") as f:
    json.dump(nested_dict, fp=f)